In [ ]:
import torch
import pyiqa
from PIL import Image
from torchvision import transforms
import glob
import os
import pandas as pd
import tempfile
from tqdm import tqdm
from dotenv import load_dotenv

In [ ]:
# -------------------------
# 文件夹路径
# -------------------------
load_dotenv("../.env")
ROOT = os.getenv("ROOT")
print("ROOT:", ROOT)
RESULTS_DIR = f"{ROOT}/output/gen_result/results_mogle_gate_1.0"
test_dir = f"{RESULTS_DIR}/result"
ref_dir = f"{ROOT}/t2v_dataset/testB"  # 真实图像（ground truth）
# -------------------------
# 获取图片
# -------------------------
ref_imgs = glob.glob(os.path.join(ref_dir, "*.png"))
test_imgs = glob.glob(os.path.join(test_dir, "*.png"))

ref_dict = {os.path.basename(p): p for p in ref_imgs}
test_dict = {os.path.basename(p): p for p in test_imgs}

common_names = sorted(set(ref_dict.keys()) & set(test_dict.keys()))

if not common_names:
    raise ValueError("两个文件夹没有任何同名图片，请检查命名！")

print(f"✅ 发现 {len(common_names)} 对匹配图片")
print(f"📁 真实图像文件夹: {ref_dir}")
print(f"📁 生成图像文件夹: {test_dir}\n")

# -------------------------
# 创建指标
# -------------------------
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🔧 使用设备: {device}\n")

metric_psnr = pyiqa.create_metric('psnr', device=device)
metric_ssim = pyiqa.create_metric('ssim', device=device)
metric_lpips = pyiqa.create_metric('lpips', device=device)
metric_fid = pyiqa.create_metric('fid', device=device)

to_tensor = transforms.ToTensor()
results = []

# -------------------------
# 定义统一的图像预处理函数（直接resize）
# -------------------------
def preprocess_image(img_path, target_size=256):
    """
    统一的图像预处理：直接resize → 转tensor
    （与训练脚本保持一致）
    """
    img = Image.open(img_path).convert('RGB')
    
    # 直接resize到固定尺寸
    img = img.resize((target_size, target_size), Image.Resampling.LANCZOS)
    
    # 转tensor，值域 [0, 1]
    tensor = to_tensor(img)  # 自动转为 [0, 1]
    
    return tensor.unsqueeze(0)

# -------------------------
# PSNR / SSIM / LPIPS 逐对计算
# -------------------------
print("📊 开始计算 PSNR/SSIM/LPIPS...")
with torch.no_grad():
    for i, name in enumerate(tqdm(common_names, desc="Processing images")):
        ref_path = ref_dict[name]
        test_path = test_dict[name]
        
        # 统一预处理两个图像
        ref_tensor = preprocess_image(ref_path).to(device)
        test_tensor = preprocess_image(test_path).to(device)
        
        # 计算指标（生成图像 vs 真实图像）
        psnr = metric_psnr(test_tensor, ref_tensor).item()
        ssim = metric_ssim(test_tensor, ref_tensor).item()
        lpips = metric_lpips(test_tensor, ref_tensor).item()
        
        results.append({
            "filename": name,
            "PSNR": psnr,
            "SSIM": ssim,
            "LPIPS": lpips,
        })
        
        # 及时清理中间变量和显存
        del ref_tensor, test_tensor
        if (i + 1) % 100 == 0:
            torch.cuda.empty_cache()

# -------------------------
# 批量计算 FID（仅针对匹配图片）
# -------------------------
print("\n📊 开始计算 FID...")
with tempfile.TemporaryDirectory() as tmpdir:
    tmp_ref = os.path.join(tmpdir, "ref")
    tmp_test = os.path.join(tmpdir, "test")
    os.makedirs(tmp_ref, exist_ok=True)
    os.makedirs(tmp_test, exist_ok=True)
    
    # 复制并处理匹配的图片到临时文件夹
    for name in tqdm(common_names, desc="Copying images for FID"):
        ref_img = Image.open(ref_dict[name]).convert('RGB')
        test_img = Image.open(test_dict[name]).convert('RGB')
        
        # 直接resize处理（与训练脚本保持一致）
        ref_img = ref_img.resize((256, 256), Image.Resampling.LANCZOS)
        test_img = test_img.resize((256, 256), Image.Resampling.LANCZOS)
        
        ref_img.save(os.path.join(tmp_ref, name))
        test_img.save(os.path.join(tmp_test, name))
    
    # 计算 FID（生成图像文件夹 vs 真实图像文件夹）
    fid_value = metric_fid(tmp_test, tmp_ref).item()

# -------------------------
# 保存结果
# -------------------------
df = pd.DataFrame(results)

# 添加平均值行
avg_row = {
    'filename': 'Average',
    'PSNR': df['PSNR'].mean(),
    'SSIM': df['SSIM'].mean(),
    'LPIPS': df['LPIPS'].mean(),
}
df = pd.concat([df, pd.DataFrame([avg_row])], ignore_index=True)

# 保存 CSV
csv_path = RESULTS_DIR+"/metric_results.csv"
df.to_csv(csv_path, index=False)


# -------------------------
# 打印结果摘要
# -------------------------
print("\n" + "="*60)
print("✅ 计算完成！")
print("="*60)
print(f"📊 逐图指标已保存到: {csv_path}")
print(f"\n📈 平均指标:")
print(f"   PSNR:  {avg_row['PSNR']:.4f} dB  (↑ 越高越好) || 29.45")
print(f"   SSIM:  {avg_row['SSIM']:.4f}     (↑ 越高越好, 最佳=1) || 0.7504")
print(f"   LPIPS: {avg_row['LPIPS']:.4f}     (↓ 越低越好) || 0.1650")
print(f"   FID:   {fid_value:.4f}     (↓ 越低越好) || 34.02")
print("="*60)

# -------------------------
# 保存对比图片到文件夹
# -------------------------
print("\n📸 开始保存对比图片...")
output_compare_dir = RESULTS_DIR+"/comparison"

# 清空输出文件夹
if os.path.exists(output_compare_dir):
    import shutil
    shutil.rmtree(output_compare_dir)
    print(f"✓ 已清空原有的 {output_compare_dir} 文件夹")

os.makedirs(output_compare_dir, exist_ok=True)

for name in tqdm(common_names, desc="Saving comparison images"):
    ref_path = ref_dict[name]
    test_path = test_dict[name]
    
    ref_img = Image.open(ref_path).convert('RGB')
    test_img = Image.open(test_path).convert('RGB')
    
    # 直接resize处理
    ref_img = ref_img.resize((256, 256))
    test_img = test_img.resize((256, 256))
    
    # 创建对比图 (真实 | 生成)
    compare_img = Image.new('RGB', (512, 256))
    compare_img.paste(ref_img, (0, 0))
    compare_img.paste(test_img, (256, 0))
    
    # 获取对应的指标
    row = df[df['filename'] == name].iloc[0]
    psnr = row['PSNR']
    ssim = row['SSIM']
    lpips = row['LPIPS']
    
    # 保存文件名包含指标信息
    output_name = f"{os.path.splitext(name)[0]}_PSNR{psnr:.2f}_SSIM{ssim:.4f}_LPIPS{lpips:.4f}.png"
    output_path = os.path.join(output_compare_dir, output_name)
    compare_img.save(output_path)

print(f"✅ 对比图片已保存到: {output_compare_dir}")


✅ 发现 2 对匹配图片
📁 真实图像文件夹: /home/cunjian/kai/cache/T2V/t2v_dataset/testB
📁 生成图像文件夹: /home/cunjian/kai/cache/T2V/output/gen_result/results_mogle_gate_1.0/result

🔧 使用设备: cuda

Loading pretrained model LPIPS from /home/cunjian/.cache/torch/hub/pyiqa/LPIPS_v0.1_alex-df73285e.pth
Loading pretrained model Inception3 from /home/cunjian/.cache/torch/hub/pyiqa/pt_inception-2015-12-05-6726825d.pth
📊 开始计算 PSNR/SSIM/LPIPS...


Processing images: 100%|██████████| 2/2 [00:00<00:00, 58.82it/s]



📊 开始计算 FID...


Copying images for FID: 100%|██████████| 2/2 [00:00<00:00, 10.19it/s]


Computing FID between two folders
Found 2 images in the folder /tmp/tmp76dtcb3j/test


...

Found 2 images in the folder /tmp/tmp76dtcb3j/ref


FID ref: 100%|██████████| 1/1 [00:00<00:00,  3.52it/s]



✅ 计算完成！
📊 逐图指标已保存到: /home/cunjian/kai/cache/T2V/output/gen_result/results_mogle_gate_1.0/metric_results.csv

📈 平均指标:
   PSNR:  22.4485 dB  (↑ 越高越好) || 29.45
   SSIM:  0.7656     (↑ 越高越好, 最佳=1) || 0.7504
   LPIPS: 0.1416     (↓ 越低越好) || 0.1650
   FID:   70.3302     (↓ 越低越好) || 34.02

📸 开始保存对比图片...


Saving comparison images: 100%|██████████| 2/2 [00:00<00:00,  9.09it/s]

✅ 对比图片已保存到: /home/cunjian/kai/cache/T2V/output/gen_result/results_mogle_gate_1.0/comparison


📈 平均指标:
   PSNR:  21.3718 dB  (↑ 越高越好) || 29.45
   SSIM:  0.7871     (↑ 越高越好, 最佳=1) || 0.7504
   LPIPS: 0.1698     (↓ 越低越好) || 0.1650
   FID:   35.0483     (↓ 越低越好) || 34.02

📈 平均指标:
   PSNR:  21.4603 dB  (↑ 越高越好) || 29.45
   SSIM:  0.7907     (↑ 越高越好, 最佳=1) || 0.7504
   LPIPS: 0.1671     (↓ 越低越好) || 0.1650
   FID:   34.9702     (↓ 越低越好) || 34.02

📈 平均指标:
   PSNR:  21.4549 dB  (↑ 越高越好) || 29.45
   SSIM:  0.7903     (↑ 越高越好, 最佳=1) || 0.7504
   LPIPS: 0.1673     (↓ 越低越好) || 0.1650
   FID:   34.4538     (↓ 越低越好) || 34.02

📈 平均指标:
   PSNR:  20.8287 dB  (↑ 越高越好)
   SSIM:  0.7597     (↑ 越高越好, 最佳=1)
   LPIPS: 0.2049     (↓ 越低越好)
   FID:   36.5074     (↓ 越低越好)
   
📈 平均指标:
   PSNR:  21.5972 dB  (↑ 越高越好) || 29.45
   SSIM:  0.7862     (↑ 越高越好, 最佳=1) || 0.7504
   LPIPS: 0.1676     (↓ 越低越好) || 0.1650
   FID:   20.3711     (↓ 越低越好) || 34.02
============================================

📈 平均指标:
   PSNR:  22.3148 dB  (↑ 越高越好) || 29.45
   SSIM:  0.7986     (↑ 越高越好, 最佳=1) || 0.7504
   LPIPS: 0.1616     (↓ 越低越好) || 0.1650
   FID:   18.2953     (↓ 越低越好) || 34.02
============================================================